In [1]:
import astroquery.jplhorizons as jpl
import astropy.time as time
import astropy.units as u
import numpy as np
import pandas as pd
import urllib.request
import read_web
from datetime import datetime
import re

In [2]:
# Convert from 1958 Feb. 1 to 1958-02-01
def convert_date(date):
    if date is None:
        return "N/A"
    elif "Sept." in date:
        date = date.replace('Sept.', 'Sep.')
    elif re.search(r'\.\d', date):
        date = date.replace('.', '. ')
    try:
        date_obj = datetime.strptime(date, '%Y %b. %d').strftime('%Y-%m-%d')
    except ValueError as e:
        if any(month in str(e) for month in ("May", "June", "July")):
            date_obj = datetime.strptime(date, '%Y %B %d').strftime('%Y-%m-%d')
        else:
            return "N/A"

    return date_obj

In [6]:
missionTable = read_web.read_website()
# Send table to dataframe, taking columns from the first row
missionTable = pd.DataFrame(missionTable[1:], columns=missionTable[0])
# Uppercase all spacecraft names
missionTable['Spacecraft'] = missionTable['Spacecraft'].str.upper()

# Convert dates to astropy time objects
missionTable['Launch Date (UT)'] = [convert_date(date) for date in missionTable['Launch Date (UT)']]

# Set spacecraft name in 75-78 as MMS 1-4
missionTable.loc[75:78, 'Spacecraft'] = ['MMS 1', 'MMS 2', 'MMS 3', 'MMS 4']

# If spacecraft contains CLUSTER, split at space and take second element
missionTable.loc[missionTable['Spacecraft'].str.contains('CLUSTER'), 'Spacecraft'] = \
    missionTable.loc[missionTable['Spacecraft'].str.contains('CLUSTER'), 'Spacecraft'].str.split().str[1]

# 
clusterDict = {"SAMBA": "CLUSTER II-FM7 (SAMBA)", "SALSA": "CLUSTER II-FM6 (SALSA)", "TANGO": "CLUSTER II-FM8 (TANGO)", "RUMBA": "CLUSTER II-FM5 (RUMBA)"}
missionTable['Spacecraft'] = missionTable['Spacecraft'].replace(clusterDict)

# If no launch date, remove
missionTable = missionTable[~missionTable['Launch Date (UT)'].str.contains("N/A")]

# If "fail" in second column, remove row
missionTable = missionTable[~missionTable.iloc[:,1].str.contains("fail")]

#missionTable[25:]

In [3]:
# Import satcat.csv from Celestrak
# https://celestrak.org/pub/satcat.csv

# First download the latest satcat.csv file from Celestrak
# https://celestrak.com/pub/satcat.csv

url = 'https://celestrak.com/pub/satcat.csv'
filename = 'satcat.csv'
urllib.request.urlretrieve(url, filename)

('satcat.csv', <http.client.HTTPMessage at 0x255b5187ca0>)

In [7]:
# Read satcat.csv into a pandas dataframe
satcat = pd.read_csv(filename)

In [8]:
missionList = missionTable["Spacecraft"]
spacecraft = missionList

In [10]:
# Get lines with spacecraft names from satcat
# and add to new dataframe
spacecraft = [craft.upper() for craft in spacecraft]
mask = satcat['OBJECT_NAME'].isin(spacecraft)
new_df = satcat[mask]
print(len(new_df))
new_df

37


,OBJECT_NAME,OBJECT_ID,NORAD_CAT_ID,OBJECT_TYPE,OPS_STATUS_CODE,OWNER,LAUNCH_DATE,LAUNCH_SITE,DECAY_DATE,PERIOD,INCLINATION,APOGEE,PERIGEE,RCS,DATA_STATUS_CODE,ORBIT_CENTER,ORBIT_TYPE
3,EXPLORER 1,1958-001A,4,PAY,D,US,1958-02-01,AFETR,1970-03-31,88.48,33.15,215.0,183.0,NaN,NaN,EA,IMP
5,EXPLORER 3,1958-003A,6,PAY,D,US,1958-03-26,AFETR,1958-06-28,103.60,33.50,1739.0,117.0,NaN,NaN,EA,IMP
8,EXPLORER 4,1958-005A,9,PAY,D,US,1958-07-26,AFETR,1959-10-23,92.81,50.25,585.0,239.0,NaN,NaN,EA,IMP
21,EXPLORER 7,1959-009A,22,PAY,NaN,US,1959-10-13,AFETR,NaN,95.86,50.28,646.0,475.0,0.5003,NaN,EA,ORB
109,PIONEER 1,1958-007A,110,PAY,D,US,1958-10-11,AFETR,1958-10-12,NaN,NaN,NaN,NaN,NaN,NIE,EA,IMP
110,PIONEER 3,1958-008A,111,PAY,D,US,1958-12-06,AFETR,1958-12-07,NaN,NaN,NaN,NaN,NaN,NIE,EA,IMP
112,PIONEER 4,1959-013A,113,PAY,NaN,US,1959-03-03,AFETR,NaN,NaN,NaN,NaN,NaN,NaN,NEA,SU,ORB
169,EXPLORER 12,1961-020A,170,PAY,D,US,1961-08-15,AFETR,1963-09-30,1594.62,33.83,76977.0,693.0,NaN,NaN,EA,IMP
204,TRAAC,1961-031B,205,PAY,NaN,US,1961-11-15,AFETR,NaN,105.69,32.44,1102.0,951.0,0.5519,NaN,EA,ORB
246,DISCOVERER 38,1962-005A,247,PAY,D,US,1962-02-27,AFWTR,1962-03-21,88.32,82.23,201.0,182.0,NaN,NaN,EA,IMP


In [11]:
# Move active spacecraft to new dataframe
# judging activity by the "DECAY_DATE" column
# if the spacecraft has not decayed, it is active

active_df = new_df[new_df['DECAY_DATE'].isnull()]
print(len(active_df))
active_df

25


,OBJECT_NAME,OBJECT_ID,NORAD_CAT_ID,OBJECT_TYPE,OPS_STATUS_CODE,OWNER,LAUNCH_DATE,LAUNCH_SITE,DECAY_DATE,PERIOD,INCLINATION,APOGEE,PERIGEE,RCS,DATA_STATUS_CODE,ORBIT_CENTER,ORBIT_TYPE
21,EXPLORER 7,1959-009A,22,PAY,NaN,US,1959-10-13,AFETR,NaN,95.86,50.28,646.0,475.0,0.5003,NaN,EA,ORB
112,PIONEER 4,1959-013A,113,PAY,NaN,US,1959-03-03,AFETR,NaN,NaN,NaN,NaN,NaN,NaN,NEA,SU,ORB
204,TRAAC,1961-031B,205,PAY,NaN,US,1961-11-15,AFETR,NaN,105.69,32.44,1102.0,951.0,0.5519,NaN,EA,ORB
373,MARINER 2,1962-041A,374,PAY,NaN,US,1962-08-27,AFETR,NaN,NaN,NaN,NaN,NaN,0.0005,NEA,SU,ORB
937,MARINER 4,1964-077A,938,PAY,NaN,US,1964-11-28,AFETR,NaN,NaN,NaN,NaN,NaN,NaN,NEA,SU,ORB
2844,MARINER 5,1967-060A,2845,PAY,NaN,US,1967-06-14,AFETR,NaN,NaN,NaN,NaN,NaN,NaN,NEA,SU,ORB
5859,PIONEER 10,1972-012A,5860,PAY,NaN,US,1972-03-03,AFETR,NaN,NaN,NaN,NaN,NaN,NaN,NEA,SS,ORB
6420,PIONEER 11,1973-019A,6421,PAY,NaN,US,1973-04-06,AFETR,NaN,NaN,NaN,NaN,NaN,NaN,NEA,SS,ORB
7566,HELIOS 1,1974-097A,7567,PAY,-,GER,1974-12-10,AFETR,NaN,NaN,NaN,NaN,NaN,NaN,NEA,SU,ORB
8581,HELIOS 2,1976-003A,8582,PAY,-,GER,1976-01-15,AFETR,NaN,NaN,NaN,NaN,NaN,NaN,NEA,SU,ORB


In [12]:
# Remove all columns except "OBJECT_NAME" "OBJECT_ID" "NORAD_CAT_ID" "LAUNCH_DATE" "OWNER" "ORBIT_CENTER"
active_df = active_df[['OBJECT_NAME', 'OBJECT_ID', 'NORAD_CAT_ID', 'LAUNCH_DATE', 'OWNER', 'ORBIT_CENTER']]
active_df

,OBJECT_NAME,OBJECT_ID,NORAD_CAT_ID,LAUNCH_DATE,OWNER,ORBIT_CENTER
21,EXPLORER 7,1959-009A,22,1959-10-13,US,EA
112,PIONEER 4,1959-013A,113,1959-03-03,US,SU
204,TRAAC,1961-031B,205,1961-11-15,US,EA
373,MARINER 2,1962-041A,374,1962-08-27,US,SU
937,MARINER 4,1964-077A,938,1964-11-28,US,SU
2844,MARINER 5,1967-060A,2845,1967-06-14,US,SU
5859,PIONEER 10,1972-012A,5860,1972-03-03,US,SS
6420,PIONEER 11,1973-019A,6421,1973-04-06,US,SS
7566,HELIOS 1,1974-097A,7567,1974-12-10,GER,SU
8581,HELIOS 2,1976-003A,8582,1976-01-15,GER,SU
